# Exploration of DrugMechDB

Here, I explore [DrugMechDB](https://sulab.github.io/DrugMechDB/) which could ideally provide us with ture positive paths / MOAs.

In [1]:
import pandas as pd
import os.path as osp
from collections import Counter
from urllib import request
import json

## Load in the DrugMechDB file:

In [2]:
VALIDATION_DIR = '../data/validation'

The database download comes as an excel file with many separate sheets, so it needs to be read in as separate dataframes:

In [3]:
dm_db = pd.ExcelFile(osp.join(VALIDATION_DIR, 'indication_MOA_paths.xlsx'))

In [4]:
indications = pd.read_excel(dm_db, 'sample_indications')

In [5]:
indications.head()

,name,num_ind_dc,db_id,comp_mesh_ids,disease_name,dis_mesh_id,Comments
0,imatinib,11.0,DB00619,MESH:D000068877,CML (ph+),MESH:D015464,"Multiple ways to represent this path, experime..."
1,imatinib,11.0,DB00619,MESH:D000068877,Systemic mast cell disease,MESH:D034721,NaN
2,imatinib,11.0,DB00619,MESH:D000068877,Systemic mast cell disease,MESH:D034721,NaN
3,acetaminophen,NaN,DB00316,MESH:D000082,Pain,MESH:D010146,"Multiple ways to represent this path, experime..."
4,acetaminophen,NaN,DB00316,MESH:D000082,Pain,MESH:D010146,NaN


We can see the the drugs are provided as [DrugBank](https://go.drugbank.com) identifiers, and the diseases are providued as [MeSh identifiers](https://www.nlm.nih.gov/mesh/meshhome.html). 

Let's look at the paths:

## Path Instances in DrugMechDB

In [6]:
paths = pd.read_excel(dm_db, 'paths')

In [7]:
paths.head()

,n1,e1,n2,e2,n3,e3,n4,e4,n5,e5,n6,e6,n7,e7,n8
0,imatinib,INHIBITS,BCR/ABL,CAUSES,CML (ph+),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,imatinib,INHIBITS,c-Kit,UP_REGULATES,Cellular proliferation,CAUSES,Systemic mast cell disease,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,imatinib,INHIBITS,Pdgf,UP_REGULATES,Cellular proliferation,CAUSES,Systemic mast cell disease,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,acetaminophen,INHIBITS,cycloxygenaze pathways,ASSOCIATED_WITH,Pain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,acetaminophen,INHIBITS,Cox-1,PRODUCES,Prostaglandins,CAUSES,Pain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


How long are most of the paths? I'll define the length of the paths as the number of edges/hops.

In [8]:
Counter([(row.count() - 1) / 2 for i, row in paths.iterrows()])

Counter({4.0: 59, 3.0: 38, 2.0: 13, 5.0: 8, 1.0: 3, 7.0: 1, 6.0: 1})

It looks like most are length 4.

Only 3 are above length 5.

What if we only take those which end in a biological process?

It might be better to work from the JSON file instead, which can be found [here](https://github.com/SuLab/DrugMechDB/blob/1.0/indication_paths.json). In contrast to the paths in the dataframe, the JSON version has the types for each node.

In [9]:
url = "https://raw.githubusercontent.com/SuLab/DrugMechDB/1.0/indication_paths.json"
filepath = osp.join(VALIDATION_DIR, "drugmechdb_paths.json")

In [10]:
with request.urlopen(url) as fl:
    data_json = json.loads(fl.read())
    #json.dump(dict(data_json), filepath)
    #print(data_json)

Are there the same number of paths as before?

In [11]:
len(data_json)

123

In [12]:
len(data_json) == len(paths)

True

Let's take a look at an example path:

In [13]:
data_json[1]

{'directed': True,
 'multigraph': True,
 'graph': {'drug': 'imatinib',
  'disease': 'Systemic mast cell disease',
  'drugbank': 'DB00619',
  'drug_mesh': 'MESH:D000068877',
  'disease_mesh': 'MESH:D034721'},
 'nodes': [{'id': 'MESH:D000068877', 'name': 'imatinib', 'label': 'Drug'},
  {'id': 'UniProt:P10721', 'name': 'c-Kit', 'label': 'Protein'},
  {'id': 'GO:0008283',
   'name': 'Cellular proliferation',
   'label': 'Biological Process'},
  {'id': 'MESH:D034721',
   'name': 'Systemic mast cell disease',
   'label': 'Disease'}],
 'links': [{'source': 'MESH:D000068877',
   'target': 'UniProt:P10721',
   'key': 'INHIBITS'},
  {'source': 'UniProt:P10721', 'target': 'GO:0008283', 'key': 'UP_REGULATES'},
  {'source': 'GO:0008283', 'target': 'MESH:D034721', 'key': 'CAUSES'}]}

We can get the number of nodes involved:

In [14]:
len(data_json[1]['nodes'])

4

... and using that, get the second-to-last node (the last is always a disease).

In [15]:
data_json[1]['nodes'][len(data_json[1]['nodes']) - 2]

{'id': 'GO:0008283',
 'name': 'Cellular proliferation',
 'label': 'Biological Process'}

Cool, let's create a new object containing only those which end in a BP node:

In [16]:
bp_paths = []
bp_path_lengths = []
for path in data_json:
    # if second to last node is BP,
    if path['nodes'][len(path['nodes']) - 2]['label'] == 'Biological Process':
        bp_paths.append(path)
        bp_path_lengths.append(len(path['links']))

How many of the 162 paths end in a BP?

In [17]:
len(bp_paths)

62

In [18]:
Counter(bp_path_lengths)

Counter({3: 25, 4: 24, 5: 7, 2: 6})